In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def essBoundaryFun(x):
    return 0.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.setNaturalRHS(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)
rhs.setRhsStencil(mesh, K)

In [5]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, ksp)

In [7]:
trainingData = dta.StokesData(range(32))
trainingData.readData(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshapeInputImg()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [9]:
steps = int(5)
for s in range(steps):
    print('step = ', s)
    batchSamples = torch.LongTensor(model.batchSizeN).random_(0, trainingData.nSamples)
    model.pfStep(batchSamples)

step =  0
loss =  tensor(5875348.5000, grad_fn=<SubBackward0>)
step =  1
loss =  tensor(5874116., grad_fn=<SubBackward0>)
step =  2
loss =  tensor(5868407., grad_fn=<SubBackward0>)
step =  3
loss =  tensor(5867252., grad_fn=<SubBackward0>)
step =  4
loss =  tensor(5867238.5000, grad_fn=<SubBackward0>)
step =  5
loss =  tensor(5866145., grad_fn=<SubBackward0>)
step =  6
loss =  tensor(5861843., grad_fn=<SubBackward0>)
step =  7
loss =  tensor(5862414., grad_fn=<SubBackward0>)
step =  8
loss =  tensor(5860766., grad_fn=<SubBackward0>)
step =  9
loss =  tensor(5857771., grad_fn=<SubBackward0>)
step =  10
loss =  tensor(5855629., grad_fn=<SubBackward0>)
step =  11
loss =  tensor(5857502.5000, grad_fn=<SubBackward0>)
step =  12
loss =  tensor(5855002., grad_fn=<SubBackward0>)
step =  13
loss =  tensor(5852093., grad_fn=<SubBackward0>)
step =  14
loss =  tensor(5852227.5000, grad_fn=<SubBackward0>)
step =  15
loss =  tensor(5849658.5000, grad_fn=<SubBackward0>)
step =  16
loss =  tensor(5849

step =  134
loss =  tensor(5617319., grad_fn=<SubBackward0>)
step =  135
loss =  tensor(5620020., grad_fn=<SubBackward0>)
step =  136
loss =  tensor(5623093., grad_fn=<SubBackward0>)
step =  137
loss =  tensor(5613890., grad_fn=<SubBackward0>)
step =  138
loss =  tensor(5624133.5000, grad_fn=<SubBackward0>)
step =  139
loss =  tensor(5619156., grad_fn=<SubBackward0>)
step =  140
loss =  tensor(5610193.5000, grad_fn=<SubBackward0>)
step =  141
loss =  tensor(5606429., grad_fn=<SubBackward0>)
step =  142
loss =  tensor(5608740., grad_fn=<SubBackward0>)
step =  143
loss =  tensor(5616539., grad_fn=<SubBackward0>)
step =  144
loss =  tensor(5605237., grad_fn=<SubBackward0>)
step =  145
loss =  tensor(5610547., grad_fn=<SubBackward0>)
step =  146
loss =  tensor(5599496.5000, grad_fn=<SubBackward0>)
step =  147
loss =  tensor(5596762., grad_fn=<SubBackward0>)
step =  148
loss =  tensor(5596705., grad_fn=<SubBackward0>)
step =  149
loss =  tensor(5594619.5000, grad_fn=<SubBackward0>)
step =  

step =  266
loss =  tensor(5508691., grad_fn=<SubBackward0>)
step =  267
loss =  tensor(5485025., grad_fn=<SubBackward0>)
step =  268
loss =  tensor(5470435., grad_fn=<SubBackward0>)
step =  269
loss =  tensor(5483238., grad_fn=<SubBackward0>)
step =  270
loss =  tensor(5491588., grad_fn=<SubBackward0>)
step =  271
loss =  tensor(5491281., grad_fn=<SubBackward0>)
step =  272
loss =  tensor(5497464.5000, grad_fn=<SubBackward0>)
step =  273
loss =  tensor(5485090., grad_fn=<SubBackward0>)
step =  274
loss =  tensor(5499498.5000, grad_fn=<SubBackward0>)
step =  275
loss =  tensor(5483603., grad_fn=<SubBackward0>)
step =  276
loss =  tensor(5484888., grad_fn=<SubBackward0>)
step =  277
loss =  tensor(5499669., grad_fn=<SubBackward0>)
step =  278
loss =  tensor(5488709., grad_fn=<SubBackward0>)
step =  279
loss =  tensor(5473951.5000, grad_fn=<SubBackward0>)
step =  280
loss =  tensor(5509757., grad_fn=<SubBackward0>)
step =  281
loss =  tensor(5493258., grad_fn=<SubBackward0>)
step =  282


step =  399
loss =  tensor(5494078.5000, grad_fn=<SubBackward0>)
step =  400
loss =  tensor(5479059.5000, grad_fn=<SubBackward0>)
step =  401
loss =  tensor(5493719., grad_fn=<SubBackward0>)
step =  402
loss =  tensor(5502052., grad_fn=<SubBackward0>)
step =  403
loss =  tensor(5494269., grad_fn=<SubBackward0>)
step =  404
loss =  tensor(5493135.5000, grad_fn=<SubBackward0>)
step =  405
loss =  tensor(5495119., grad_fn=<SubBackward0>)
step =  406
loss =  tensor(5493347., grad_fn=<SubBackward0>)
step =  407
loss =  tensor(5479389.5000, grad_fn=<SubBackward0>)
step =  408
loss =  tensor(5523333., grad_fn=<SubBackward0>)
step =  409
loss =  tensor(5492214., grad_fn=<SubBackward0>)
step =  410
loss =  tensor(5488806., grad_fn=<SubBackward0>)
step =  411
loss =  tensor(5508482.5000, grad_fn=<SubBackward0>)
step =  412
loss =  tensor(5513101., grad_fn=<SubBackward0>)
step =  413
loss =  tensor(5489691., grad_fn=<SubBackward0>)
step =  414
loss =  tensor(5499874.5000, grad_fn=<SubBackward0>)


step =  532
loss =  tensor(5508096.5000, grad_fn=<SubBackward0>)
step =  533
loss =  tensor(5484493., grad_fn=<SubBackward0>)
step =  534
loss =  tensor(5497860., grad_fn=<SubBackward0>)
step =  535
loss =  tensor(5490672.5000, grad_fn=<SubBackward0>)
step =  536
loss =  tensor(5478140., grad_fn=<SubBackward0>)
step =  537
loss =  tensor(5510404., grad_fn=<SubBackward0>)
step =  538
loss =  tensor(5495801., grad_fn=<SubBackward0>)
step =  539
loss =  tensor(5473873., grad_fn=<SubBackward0>)
step =  540
loss =  tensor(5514202., grad_fn=<SubBackward0>)
step =  541
loss =  tensor(5519607., grad_fn=<SubBackward0>)
step =  542
loss =  tensor(5496276.5000, grad_fn=<SubBackward0>)
step =  543
loss =  tensor(5489102., grad_fn=<SubBackward0>)
step =  544
loss =  tensor(5491633.5000, grad_fn=<SubBackward0>)
step =  545
loss =  tensor(5486913.5000, grad_fn=<SubBackward0>)
step =  546
loss =  tensor(5487492., grad_fn=<SubBackward0>)
step =  547
loss =  tensor(5499970., grad_fn=<SubBackward0>)
step

...

loss =  tensor(5468048., grad_fn=<SubBackward0>)
step =  1711
loss =  tensor(5464315., grad_fn=<SubBackward0>)
step =  1712
loss =  tensor(5470807., grad_fn=<SubBackward0>)
step =  1713
loss =  tensor(5474448.5000, grad_fn=<SubBackward0>)
step =  1714
loss =  tensor(5482260.5000, grad_fn=<SubBackward0>)
step =  1715
loss =  tensor(5456609., grad_fn=<SubBackward0>)
step =  1716
loss =  tensor(5463645., grad_fn=<SubBackward0>)
step =  1717
loss =  tensor(5492184., grad_fn=<SubBackward0>)
step =  1718
loss =  tensor(5472189.5000, grad_fn=<SubBackward0>)
step =  1719
loss =  tensor(5467976.5000, grad_fn=<SubBackward0>)
step =  1720
loss =  tensor(5469462., grad_fn=<SubBackward0>)
step =  1721
loss =  tensor(5472299., grad_fn=<SubBackward0>)
step =  1722
loss =  tensor(5499162., grad_fn=<SubBackward0>)
step =  1723
loss =  tensor(5475487., grad_fn=<SubBackward0>)
step =  1724
loss =  tensor(5469556.5000, grad_fn=<SubBackward0>)
step =  1725
loss =  tensor(5470111., grad_fn=<SubBackward0>)
s

loss =  tensor(5486779., grad_fn=<SubBackward0>)
step =  1841
loss =  tensor(5473619.5000, grad_fn=<SubBackward0>)
step =  1842
loss =  tensor(5478713., grad_fn=<SubBackward0>)
step =  1843
loss =  tensor(5462794., grad_fn=<SubBackward0>)
step =  1844
loss =  tensor(5487309., grad_fn=<SubBackward0>)
step =  1845
loss =  tensor(5464577., grad_fn=<SubBackward0>)
step =  1846
loss =  tensor(5473805., grad_fn=<SubBackward0>)
step =  1847
loss =  tensor(5474328., grad_fn=<SubBackward0>)
step =  1848
loss =  tensor(5502307.5000, grad_fn=<SubBackward0>)
step =  1849
loss =  tensor(5458349., grad_fn=<SubBackward0>)
step =  1850
loss =  tensor(5460260., grad_fn=<SubBackward0>)
step =  1851
loss =  tensor(5470009.5000, grad_fn=<SubBackward0>)
step =  1852
loss =  tensor(5478433., grad_fn=<SubBackward0>)
step =  1853
loss =  tensor(5470932., grad_fn=<SubBackward0>)
step =  1854
loss =  tensor(5516898., grad_fn=<SubBackward0>)
step =  1855
loss =  tensor(5462550., grad_fn=<SubBackward0>)
step =  1

step =  1971
loss =  tensor(5480658., grad_fn=<SubBackward0>)
step =  1972
loss =  tensor(5469067., grad_fn=<SubBackward0>)
step =  1973
loss =  tensor(5474743., grad_fn=<SubBackward0>)
step =  1974
loss =  tensor(5475393.5000, grad_fn=<SubBackward0>)
step =  1975
loss =  tensor(5467252.5000, grad_fn=<SubBackward0>)
step =  1976
loss =  tensor(5479812., grad_fn=<SubBackward0>)
step =  1977
loss =  tensor(5456071.5000, grad_fn=<SubBackward0>)
step =  1978
loss =  tensor(5477084., grad_fn=<SubBackward0>)
step =  1979
loss =  tensor(5458849., grad_fn=<SubBackward0>)
step =  1980
loss =  tensor(5477664.5000, grad_fn=<SubBackward0>)
step =  1981
loss =  tensor(5482575., grad_fn=<SubBackward0>)
step =  1982
loss =  tensor(5468063., grad_fn=<SubBackward0>)
step =  1983
loss =  tensor(5439619.5000, grad_fn=<SubBackward0>)
step =  1984
loss =  tensor(5479624., grad_fn=<SubBackward0>)
step =  1985
loss =  tensor(5489068., grad_fn=<SubBackward0>)
step =  1986
loss =  tensor(5473040.5000, grad_fn=

loss =  tensor(5510453., grad_fn=<SubBackward0>)
step =  2102
loss =  tensor(5490676.5000, grad_fn=<SubBackward0>)
step =  2103
loss =  tensor(5498523., grad_fn=<SubBackward0>)
step =  2104
loss =  tensor(5502348., grad_fn=<SubBackward0>)
step =  2105
loss =  tensor(5455421., grad_fn=<SubBackward0>)
step =  2106
loss =  tensor(5456056., grad_fn=<SubBackward0>)
step =  2107
loss =  tensor(5436509., grad_fn=<SubBackward0>)
step =  2108
loss =  tensor(5462923., grad_fn=<SubBackward0>)
step =  2109
loss =  tensor(5488172.5000, grad_fn=<SubBackward0>)
step =  2110
loss =  tensor(5460713., grad_fn=<SubBackward0>)
step =  2111
loss =  tensor(5436325., grad_fn=<SubBackward0>)
step =  2112
loss =  tensor(5473313., grad_fn=<SubBackward0>)
step =  2113
loss =  tensor(5470907.5000, grad_fn=<SubBackward0>)
step =  2114
loss =  tensor(5474349., grad_fn=<SubBackward0>)
step =  2115
loss =  tensor(5476775., grad_fn=<SubBackward0>)
step =  2116
loss =  tensor(5492726., grad_fn=<SubBackward0>)
step =  2

KeyboardInterrupt: 

In [ ]:
model.writer.close()
model.plotInputReconstruction()

In [ ]:
f = plt.gcf()
f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)

In [ ]:
# from torch.utils.tensorboard import SummaryWriter

# writer = SummaryWriter('runs/gendrrom')

# z = torch.randn(model.batchSizeN, model.batchSizeZ, model.dim_z)
# writer.add_graph(model.pfNet, (z, model.data.imgX))

# writer.close()

In [ ]:
# z = torch.randn(model.batchSizeN, model.batchSizeZ, model.dim_z)
# pred = model.pfNet(z, model.data.imgX)
# batchSamples = torch.LongTensor(model.batchSizeN).random_(0, trainingData.nSamples)
# loss = model.loss_pf(pred, batchSamples)

In [ ]:
loss